Here, a real PI-RADS perfomance of ALTA Klinik is evaluated against biosies

In [29]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [30]:
pirads_df = pd.read_excel("PI-RADS_bis_26_01_2024.xlsx")
pirads_df.rename(columns={"Pat-ID": "ALTAPatientID", 
                          "Befunddatum": "StudyDate"}, inplace=True)

class_df = pd.read_csv("prostate_class_dataset_demography_final_psa_vol_20240919_feat.csv", sep=";")
class_df = class_df[["ALTAPatientID", "StudyDate", "study_orthanc_id", "csPCa"]]

In [31]:
class_df

,ALTAPatientID,StudyDate,study_orthanc_id,csPCa
0,12724,2012-04-18,fecf8784-dbb3c5d4-1142b881-b6990853-955cd99c,True
1,25525,2019-05-23,e5be4a92-0dea82f9-5239b10d-0b2268e3-db270b2f,True
2,32395,2022-02-03,ddd04df3-f36da29e-a7d01715-0612285d-d3551bb7,True
3,33741,2022-07-15,7f7489ee-372560ed-ce9717de-41ed19a1-96ffea0e,True
4,33864,2022-08-01,8c59be4f-2b27d497-de2df79c-90e43ad0-4c1e55af,True
...,...,...,...,...
4481,29904,2021-03-15,db00b468-fc6ca3c0-48f7087c-c2a19702-a75140da,False
4482,30108,2021-04-06,8fd1441d-befd1615-1f2f9d9d-1e925776-c46e497c,False
4483,25501,2021-05-20,6d4719d2-30a6c777-c505f9f4-e3f26ad2-cdb882c5,False
4484,15173,2013-06-27,02023b67-286ea094-2e44260c-6f302504-5f789741,False


In [32]:
pirads_df

,ID,ALTAPatientID,StudyDate,Befundname,PI-RADS
0,1,23967,2018-11-05,mrpro3_023967_051118_0001.docx,5
1,2,26172,2019-08-20,mrpro3_026172_200819_0001.docx,5
2,3,27024,2019-12-18,mrpro3_027024_181219_0001.docx,5
3,4,27340,2020-02-06,mrpro3_027340_060220_0001.docx,5
4,5,33661,2022-07-05,mrpro3_033661_050722_0001.docx,5
...,...,...,...,...,...
4566,4567,23365,2018-08-06,mrpro4_023365_060818_0001.docx,1
4567,4568,24709,2019-02-12,mrpro4_024709_120219_0001.docx,1
4568,4569,25000,2019-03-21,mrpro4_025000_210319_0001.docx,1
4569,4570,27405,2020-02-27,mrpro3_027405_270220_0001.docx,1


In [33]:
pirads_df['StudyDate'] = pd.to_datetime(pirads_df['StudyDate'])
class_df['StudyDate'] = pd.to_datetime(class_df['StudyDate'])

In [34]:
# Merge the tables on PatientID
merged_df = pd.merge(pirads_df, class_df, on='ALTAPatientID', suffixes=('_pirads', '_mri'))

# Calculate the absolute difference in days between the dates
merged_df['date_diff'] = abs((merged_df['StudyDate_pirads'] - merged_df['StudyDate_mri']).dt.days)

# Filter the merged DataFrame to include only rows where the date difference is within 30 days
merged_df = merged_df[merged_df['date_diff'] <= 30]

# Group by PatientID and keep the row with the smallest date difference
closest_studies = merged_df.loc[merged_df.groupby('ALTAPatientID')['date_diff'].idxmin()]


In [35]:
pirads_df_histo = closest_studies.reset_index()

pirads_df_histo

,index,ID,ALTAPatientID,StudyDate_pirads,Befundname,PI-RADS,StudyDate_mri,study_orthanc_id,csPCa,date_diff
0,690,1505,424,2023-04-20,mrpro4_000424_200423_0001.docx,4,2023-04-20,cbf9953c-31c097fc-c42e9922-72cfa30e-fe5eec0d,True,0
1,1421,4001,975,2019-02-18,mrpro4_000975_180219_0001.docx,3,2019-02-15,7461db89-e3571bfc-c6c14b7d-73f0061f-bf4e8ae8,False,3
2,864,1895,1075,2018-04-09,mrpro3_001075_090418_0001.docx,4,2018-04-09,71e96a45-a7ec73d8-9be36d79-bd00d430-39977599,False,0
3,933,2002,1838,2019-09-16,mrpro3_001838_160919_0001.docx,4,2019-09-16,a4eaeb07-bf437a45-949cc82c-1c9d901e-9c4147a3,True,0
4,1291,2778,2240,2023-02-10,mrpro4_002240_100223_0001.docx,4,2023-02-10,4ea523c8-e46df99f-bcdc4f7e-09cbbbf4-5dc7a367,False,0
...,...,...,...,...,...,...,...,...,...,...
1120,549,1209,41686,2023-05-16,mrpro3_041686_160523_0001.docx,5,2023-05-16,04bdbccb-5367eb6c-8d1c7cb1-cd5cbbe1-88acc928,True,0
1121,350,706,41702,2023-05-12,mrpro3_041702_120523_0001.docx,5,2023-05-11,1cb484be-26115d29-a91b6b27-e4552f04-f88701c6,True,1
1122,1221,2604,41803,2023-05-25,mrpro4_041803_250523_0001.docx,4,2023-05-25,b0c0b368-8a33de5b-c689e557-46e163b9-740715ea,False,0
1123,1073,2242,41904,2023-06-19,mrpro3_041904_190623_0001.docx,4,2023-06-19,9106818f-c8b9b92c-a280c680-3f0e8189-2b874ba8,False,0


In [36]:
# pirads_df_histo = pd.merge(class_df, pirads_df, on=['ALTAPatientID', 'StudyDate'], how='right')
# pirads_df_histo 


In [37]:
pirads_df_histo = pirads_df_histo[~pirads_df_histo["csPCa"].isna()]

In [38]:
pirads_df_histo['PI-RADS_gt_3'] = pirads_df_histo['PI-RADS'] > 3
pirads_df_histo['PI-RADS_gt_4'] = pirads_df_histo['PI-RADS'] > 4

In [39]:
pirads_df_histo[pirads_df_histo['PI-RADS_gt_3'].isna()]

,index,ID,ALTAPatientID,StudyDate_pirads,Befundname,PI-RADS,StudyDate_mri,study_orthanc_id,csPCa,date_diff,PI-RADS_gt_3,PI-RADS_gt_4


In [40]:
print("Number of csPCa cases:")
pirads_df_histo['csPCa'].value_counts()



Number of csPCa cases:


True     660
False    465
Name: csPCa, dtype: int64

In [41]:
# Determine the minimum number of cases among the classes
min_count = pirads_df_histo['csPCa'].value_counts().min()

# Randomly sample this number of cases from each class
balanced_df = pirads_df_histo.groupby('csPCa').apply(lambda x: x.sample(350, random_state=43)).reset_index(drop=True)

# Display the balanced DataFrame
print(balanced_df)
balanced_df['csPCa'].value_counts()

# note that this is not a complete list of pirads scores!
balanced_df.to_csv("prostate_test_set_pirads.csv", sep=";")


     index    ID  ALTAPatientID StudyDate_pirads  \
0      889  1927          24039       2019-05-13   
1     1454  4434          22521       2018-03-19   
2     1399  3808          34485       2022-12-22   
3       45    74          35119       2022-12-15   
4      610  1321          40542       2022-11-22   
..     ...   ...            ...              ...   
695    690  1505            424       2023-04-20   
696    419   904          32917       2022-04-06   
697     90   177          21309       2017-06-27   
698    162   314          24693       2019-02-11   
699     54    90          26901       2019-12-02   

                         Befundname  PI-RADS StudyDate_mri  \
0    mrpro3_024039_130519_0001.docx        4    2019-05-13   
1    mrpro4_022521_190318_0001.docx        2    2018-03-16   
2    mrpro4_034485_221222_0002.docx        3    2022-12-21   
3    mrpro4_035119_151222_0001.docx        5    2022-12-15   
4    mrpro3_040542_221122_0001.docx        4    2022-11-22   
.. 

In [42]:

def caluculate_metrics(actual, predicted):# Calculate precision and recall
    precision = precision_score(actual, predicted)
    recall = recall_score(actual, predicted)
    acc = accuracy_score(actual, predicted)
    cm = confusion_matrix(actual, predicted)

    # Extract true positives, false positives, true negatives, and false negatives from the confusion matrix
    tn, fp, fn, tp = cm.ravel()

    # Calculate true positive rate (recall) and false positive rate
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    print(f"Recall: {recall:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Acc: {acc:.3f}")
    print(f"True Positive Rate (TPR): {tpr:.3f}")
    print(f"False Positive Rate (FPR): {fpr:.3f}")
    print(f"tn, fp, fn, tp: {tn}, {fp}, {fn}, {tp}")
    

    print("\nConfusion Matrix:")
    print(cm)

In [43]:

# Assuming the binary columns are named 'predicted' and 'actual'
predicted = pirads_df_histo['PI-RADS_gt_3'].to_numpy().astype(int)
actual = pirads_df_histo['csPCa'].to_numpy().astype(int)
print(f"PI-RADS>3 -> csPCa ({len(actual)} samples)")
caluculate_metrics(actual, predicted)


PI-RADS>3 -> csPCa (1125 samples)
Recall: 0.985
Precision: 0.630
Acc: 0.652
True Positive Rate (TPR): 0.985
False Positive Rate (FPR): 0.819
tn, fp, fn, tp: 84, 381, 10, 650

Confusion Matrix:
[[ 84 381]
 [ 10 650]]


In [44]:
print(f"PI-RADS>4 -> csPCa ({len(actual)} samples)")

# Assuming the binary columns are named 'predicted' and 'actual'
predicted = pirads_df_histo['PI-RADS_gt_4'].to_numpy().astype(int)
actual = pirads_df_histo['csPCa'].to_numpy().astype(int)
caluculate_metrics(actual, predicted)


PI-RADS>4 -> csPCa (1125 samples)
Recall: 0.589
Precision: 0.802
Acc: 0.674
True Positive Rate (TPR): 0.589
False Positive Rate (FPR): 0.206
tn, fp, fn, tp: 369, 96, 271, 389

Confusion Matrix:
[[369  96]
 [271 389]]
